In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LinearRegression, HuberRegressor, TheilSenRegressor
from sklearn.cluster import AgglomerativeClustering, DBSCAN, MeanShift, AffinityPropagation, OPTICS
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, silhouette_score, calinski_harabasz_score,davies_bouldin_score
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def generate_clustered_dataset(dimension,total_no_samples,no_of_cluster, random_state):
    clustered_dataset, true_label, centroids = make_blobs(n_samples=total_no_samples,
                                                          n_features=dimension, 
                                                          centers=no_of_cluster,
                                                          return_centers=True, 
                                                          random_state=random_state) #check randomstate =80
    return clustered_dataset, true_label, centroids


def plot3dwithspike(width, height, title, datapoints, spikes, myLabel=None) :
    plt.figure(figsize=(width,height))
    plt.title(title, fontsize='medium')
    ax = plt.axes(projection='3d')
    ax.scatter3D(datapoints[:, 0], datapoints[:,1], datapoints[:,2], c=myLabel, marker='o',  s=15, edgecolor='k')
    ax.scatter3D(spikes[:, 0], spikes[:, 1], spikes[:, 2], s = 80, color = 'k')
    plt.show()
    
def plotDistanceMatrix(distmat, title):
    ax = plt.axes()
    sns.heatmap(distmat, ax = ax)
    ax.set_title(title)
    plt.show()
    
def plotClusteringResult(width, height, title, label, labels_by_model, clustered_dataset, spikes):
    plt.figure(figsize=(width,height))
    plt.subplots_adjust(bottom=.05, top=.9, left=.05, right=.95)
    plt.subplot(325)
    plt.title(title, fontsize='medium')
    for i in labels_by_model:
        plt.scatter(clustered_dataset[label == i , 0] , clustered_dataset[label == i , 1] , label = i)
    plt.scatter(spikes[:,0] , spikes[:,1] , s = 80, color = 'k')
    plt.legend()
    plt.show()
    
def calc_fed_euc_dist(sldm_array):
    combined_eucl = np.concatenate(sldm_array)
    # rows are s1,s2..sn while columns are datapoints
    print(combined_eucl)
    # computing the distance of distance (e.g.: meta-distance)
    # number of all samples * number of all samples
    return euclidean_distances(combined_eucl)

def regression_per_client(data, euc_dist_data_spike, regressor="Huber"):
    euc_dist_data = euclidean_distances(data).flatten()
    local_fed_dist = np.array(calc_fed_euc_dist([euc_dist_data_spike]).flatten()).reshape((-1,1))
    if regressor == "Huber":
        model = HuberRegressor().fit(local_fed_dist,euc_dist_data)
        return [model.coef_.item(),model.intercept_]
    if regressor == "Linear":
        model = LinearRegression().fit(local_fed_dist,euc_dist_data)
        return [model.coef_.item(),model.intercept_]
    if regressor == "TheilSen":
        model = TheilSenRegressor().fit(local_fed_dist,euc_dist_data)
        return [model.coef_.item(),model.intercept_]

def generate_spikes_each_participant(dataset, reduce= 1):
    dimension = dataset.shape[1]
    row_size = np.floor((np.sqrt(dimension))/reduce).astype(int) if np.floor(np.sqrt(dimension)).astype(int) < np.floor(np.sqrt(dataset.shape[0])).astype(int) else np.floor((np.sqrt(dataset.shape[0]))/reduce).astype(int) 
    generated_spikes = np.random.uniform(low=np.min(dataset, axis=0),
                                         high=np.max(dataset, axis=0),
                                         size=(row_size, dimension))
    return generated_spikes

In [3]:
clustered_dataset = pd.read_csv(r'C:\Users\christina\Documents\Thesis\Resources\Datasets\GSE84433_series_matrix.txt', comment='!', sep="\t", header=0)
clustered_dataset = clustered_dataset.T
clustered_dataset.dropna(inplace=True)
clustered_dataset, clustered_dataset.columns = clustered_dataset[1:] , clustered_dataset.iloc[0]
true_label = clustered_dataset.iloc[:,0].astype('category').cat.codes
# clustered_dataset = clustered_dataset.drop(columns="ID_REF")
# clustered_dataset = clustered_dataset.iloc[1: , :]
# clustered_dataset = clustered_dataset.to_numpy(dtype='float64')

clustered_dataset[clustered_dataset.iloc[:,0]== "GSM2235696"]
clustered_dataset.shape

(357, 48708)

In [4]:
len(true_label.unique())

32

In [5]:
D1,D2 = np.array_split(clustered_dataset, 2)
# D1 = D1.to_numpy()
# D2 = D2.to_numpy()
D2.shape

(178, 48708)

### Participant Based Computation

In [6]:
# Each participant generates random spike in points 
# which in production environment will be shared to coordinator for creating overall spike array
generated_spikes_D1 = generate_spikes_each_participant(D1)
generated_spikes_D2 = generate_spikes_each_participant(D2)

generated_spikes = np.concatenate((generated_spikes_D1, generated_spikes_D2))
generated_spikes.shape

KeyboardInterrupt: 

In [ ]:
plot3dwithspike(width=9, height=6, title= "Clustering with actual labels", datapoints = clustered_dataset, spikes=generated_spikes, myLabel=true_label)
# # rows are s1,s2..sn while columns are datapoints
euc_dist_D1_spikes = euclidean_distances(D1,generated_spikes)
# print("Spike local distance matrix of 1st participant: \n", euc_dist_D1_spikes)

# # rows are s1,s2..sn while columns are datapoints
euc_dist_D2_spikes = euclidean_distances(D2,generated_spikes)
# print("Spike local distance matrix of 2nd participant: \n", euc_dist_D2_spikes)
  
slope_intercept_D1 = regression_per_client(data= D1,
                                           euc_dist_data_spike= euc_dist_D1_spikes,
                                           regressor="Huber")
slope_intercept_D2 = regression_per_client(data= D2,
                                           euc_dist_data_spike= euc_dist_D2_spikes,
                                           regressor="Linear")



### Coordinator Based Computation

In [ ]:
def calc_fed_euc_dist(sldm_array):
    combined_eucl = np.concatenate(sldm_array)
    # rows are s1,s2..sn while columns are datapoints
    print(combined_eucl)
    # computing the distance of distance (e.g.: meta-distance)
    # number of all samples * number of all samples
    return euclidean_distances(combined_eucl)

def construct_global_Mx_Cx_matrix(MxCx,dataset_len_array):
    Mi,Ci = np.split(np.array(MxCx),2,axis=1)
    arrayMi=Mi.flatten()
    arrayCi=Ci.flatten()
    print("arrayMi: ",arrayMi)
    print("arrayCi: ",arrayCi)
    
    Mi_avg=np.average(arrayMi)
    Ci_avg=np.average(arrayCi)
    print("Average of slopes: ", Mi_avg)
    print("Average of constants: ", Ci_avg)
    print("array of number of vectors in the datasets, i.e., shape array: \n",dataset_len_array)
    
    #Placing the respective Mi of each datapoints and getting Mx matrix
    global_Mx = block_diag(*[np.full((i, i), c) for c, i in zip(arrayMi, dataset_len_array)])
    #Placing the respective Ci of each datapoints and getting Cx matrix
    global_Cx = block_diag(*[np.full((i, i), c) for c, i in zip(arrayCi, dataset_len_array)])
    print("Average of slope or coefficients i.e. Mi's: ", global_Mx)
    print("Average of constants or intercepts i.e. Ci's: ", global_Cx)
    # The zeroes in global slopes and constants matrix are replaced by Mi_avg and Ci_avg respectively 
    # They are used to calculate the predicted distance for cross-sectional data
    # For example: distance(a1,b5) where a1 and b5 belongs to different datasets
    global_Mx[global_Mx == 0] = Mi_avg
    global_Cx[global_Cx == 0] = Ci_avg
    print("Global coefficient or slope matrix: \n", global_Mx)
    print("Global constant or intercept matrix: \n", global_Cx)
    return global_Mx, global_Cx

def calc_pred_dist_matrix(global_Mx, global_fed_euc_dist, global_Cx):
    PGDM=np.add(np.multiply(global_Mx, global_fed_euc_dist),global_Cx)
    #As distance between same points is 0
    np.fill_diagonal(PGDM,0)
    # print("Predicted Global Distance Matrix: \n",PGDM)
    return PGDM

In [ ]:
global_true_euc_dist = euclidean_distances(clustered_dataset)

# https://stackoverflow.com/questions/59765712/optics-parallelism
label = OPTICS(metric='precomputed', n_jobs=-1).fit_predict(global_true_euc_dist)
#Getting unique labels
u_labels_2 = np.unique(label)
pred_label_gtdm =  np.array(label).tolist()
#MAKE A METHOD FOR PCA
pca_2d = PCA(n_components=2)
pca_3d = PCA(n_components=3)
clustered_dataset_2d = pca_2d.fit_transform(clustered_dataset)
generated_spikes_2d = pca_2d.fit_transform(generated_spikes)
clustered_dataset_3d = pca_3d.fit_transform(clustered_dataset)
generated_spikes_3d = pca_3d.fit_transform(generated_spikes)

plt.figure(figsize=(15,15))
plt.subplots_adjust(bottom=.05, top=.9, left=.05, right=.95)
plt.subplot(325)
plt.title("Clustering with true distance matrix", fontsize='medium')
for i in u_labels_2:
    plt.scatter(clustered_dataset_2d[label == i , 0] , clustered_dataset_2d[label == i , 1] , label = i)
plt.scatter(generated_spikes_2d[:,0] , generated_spikes_2d[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

plot3dwithspike(width=9, height=6, title= "Clustering with true distance matrix", datapoints = clustered_dataset_3d, spikes=generated_spikes_3d, myLabel=pred_label_gtdm)

In [ ]:
print("Adjusted Similarity score of the clustering with true distance in (%) :", adjusted_rand_score(true_label, pred_label_gtdm)*100)
print("Adjusted mutual info score of the clustering with true distance in (%) :", adjusted_mutual_info_score(true_label, pred_label_gtdm)*100)
print("F1 score after clustering with true distance:",f1_score(true_label, pred_label_gtdm, average='micro'))
# print("Silhouette Score: ",silhouette_score(global_true_euc_dist, pred_label_gtdm, metric='precomputed')) 
print("Calinski-Harabasz Score: ", calinski_harabasz_score(global_true_euc_dist, pred_label_gtdm))
print("Davies-Bouldin Score: ", davies_bouldin_score(global_true_euc_dist, pred_label_gtdm))

In [ ]:
global_fed_euc_dist = calc_fed_euc_dist([euc_dist_D1_spikes, euc_dist_D2_spikes])

label = OPTICS(metric='precomputed', n_jobs=-1).fit_predict(global_fed_euc_dist)
#Getting unique labels
u_labels_2 = np.unique(label)
pred_label_gfdm =  np.array(label).tolist()

plt.figure(figsize=(15,15))
plt.subplots_adjust(bottom=.05, top=.9, left=.05, right=.95)
plt.subplot(325)
plt.title("Clustering with globally federated distance matrix", fontsize='medium')
for i in u_labels_2:
    plt.scatter(clustered_dataset_2d[label == i , 0] , clustered_dataset_2d[label == i , 1] , label = i)
plt.scatter(generated_spikes_2d[:,0] , generated_spikes_2d[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

plot3dwithspike(width=9, height=6, title= "Clustering with globally federated distance matrix", datapoints = clustered_dataset_3d, spikes=generated_spikes_3d, myLabel=pred_label_gfdm)

In [ ]:
print("Adjusted Similarity score of the clustering with federated distance in (%) :", adjusted_rand_score(pred_label_gtdm, pred_label_gfdm)*100)
print("Adjusted mutual info score of the clustering with federated distance in (%) :", adjusted_mutual_info_score(pred_label_gtdm, pred_label_gfdm)*100)
print("F1 score after clustering with federated distance:",f1_score(true_label, pred_label_gfdm, average='micro'))
# print("Silhouette Score:  ",silhouette_score(global_fed_euc_dist, pred_label_gfdm, metric='precomputed')) 
print("Calinski-Harabasz Score: ", calinski_harabasz_score(global_fed_euc_dist, pred_label_gfdm))
print("Davies-Bouldin Score: ", davies_bouldin_score(global_fed_euc_dist, pred_label_gfdm))
# Reference: https://stackoverflow.com/questions/58069814/python-accuracy-check-giving-0-result-for-flipped-classification

In [ ]:
MxCx = []
MxCx.append(slope_intercept_D1)
MxCx.append(slope_intercept_D2)

global_Mx, global_Cx = construct_global_Mx_Cx_matrix(MxCx,[euc_dist_D1_spikes.shape[0], euc_dist_D2_spikes.shape[0]])
global_pred_euc_dist = calc_pred_dist_matrix(global_Mx, global_fed_euc_dist, global_Cx)


In [ ]:
label = OPTICS(metric='precomputed', n_jobs=-1).fit_predict(global_pred_euc_dist)
#Getting unique labels
u_labels_2 = np.unique(label)
pred_label_2 =  np.array(label).tolist()

plt.figure(figsize=(15,15))
plt.subplots_adjust(bottom=.05, top=.9, left=.05, right=.95)
plt.subplot(325)
plt.title("Clustering with predicted distance matrix", fontsize='medium')
for i in u_labels_2:
    plt.scatter(clustered_dataset_2d[label == i , 0] , clustered_dataset_2d[label == i , 1] , label = i)
plt.scatter(generated_spikes_2d[:,0] , generated_spikes_2d[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

plot3dwithspike(width=9, height=6, title= "Clustering with globally predicted distance matrix", datapoints = clustered_dataset_3d, spikes=generated_spikes_3d, myLabel=pred_label_2)

In [ ]:
print("Adjusted Similarity score of the clustering with predicted distance in (%) :", adjusted_rand_score(pred_label_gtdm, pred_label_2)*100)
print("Adjusted mutual info score of the clustering with predicted distance in (%) :", adjusted_mutual_info_score(pred_label_gtdm, pred_label_2)*100)
print("F1 score after clustering with predicted distance:",f1_score(true_label, pred_label_2, average='micro'))
# print("Calinski-Harabasz Score: ", calinski_harabasz_score(global_pred_euc_dist, pred_label_2))
# print("Davies-Bouldin Score: ", davies_bouldin_score(global_pred_euc_dist, pred_label_2))
# print("Silhouette Score: ",silhouette_score(global_pred_euc_dist, pred_label_2, metric='precomputed')) 

In [ ]:
plotDistanceMatrix(global_fed_euc_dist, title="Federated Global Distance Matrix")
plotDistanceMatrix(euclidean_distances(clustered_dataset), title="True Global Distance Matrix")
plotDistanceMatrix(global_pred_euc_dist, title="Predicted Global Distance Matrix")

In [ ]:
# Plot outputs
# fig = plt.figure(figsize=(7,5))
# ax1 = fig.add_subplot(111)

# ax1.scatter(global_fed_euc_dist.flatten(), euclidean_distances(clustered_dataset).flatten(), s=10, c='b', marker="s", label='fed_with_true')
# ax1.scatter(global_pred_euc_dist.flatten(), euclidean_distances(clustered_dataset).flatten(), s=10, c='r', marker="o", label='pred_with_true')
# plt.legend(loc='upper left')
# plt.show()

In [ ]:
print("Pearson correlation between true and predicted global matrices:", np.corrcoef(global_true_euc_dist.flatten(),global_pred_euc_dist.flatten())[0,1])
print("Pearson correlation between true and federated global matrices:", np.corrcoef(global_true_euc_dist.flatten(),global_fed_euc_dist.flatten())[0,1])
print("Pearson correlation between federated and predicted global matrices:", np.corrcoef(global_fed_euc_dist.flatten(),global_pred_euc_dist.flatten())[0,1])

In [ ]:
abs(np.nan_to_num(100-((np.linalg.norm(global_fed_euc_dist)/np.linalg.norm(global_true_euc_dist))*100)).mean())

In [ ]:
global_fed_euc_dist.sum()

### Observation:
* Clustering doesnt depend on number of clients as there distances are all aggregated and fed to the clustering as distance matrix
* For 3 dimensional data and 2 clustering it gives very good result (100% accuracy or close to 1)
* For Linear Regression, HuberRegressor for 3 dimensional data and 3-9 clustering it gives poor result (5% to 60%) 
* The accuracy of labeling decreases gradually with the increasing number of clustering defined in the dataset
* Difference between each value of actual and predicted distance matrices are in range of (0.00000001 to 3)
* Rate of difference might have some relation with the number of clusters or data samples
* Higher the dimension of the data points, less error on the predicted distance.